In [ ]:
import numpy as np
import xarray as xr
import netCDF4
import os
import datetime
import matplotlib.pyplot as plt 
from matplotlib import cm
import shutil
from datetime import datetime, timedelta
import glob
import datetime as dt
from os import path
import cartopy.crs as ccrs
import fsspec
import git 
import json
import sys
import yaml
import requests
import cdsapi
import os
from datetime import timezone
import regionmask

In [ ]:
xr.set_options(keep_attrs=True)

In [ ]:
def nofeb(file):
    nofeb=file.sel(time=~((file.time.dt.month == 2) & (file.time.dt.day == 29)))
    return nofeb

In [ ]:
# this gets the latest hourly era5 data
def get_era5_hourly(startyear,area,variables,wd="/Volumes/hd/era5/forecast/"):
    os.chdir(wd)
    c = cdsapi.Client()
    today= datetime.today()
    year=today.year
    year_string=str(year)
    year_string_today=year_string
    most_recent_data=today - timedelta(days=7)# the release delay is less than 7 days, but just to be on the safe side
    most_recent_month=most_recent_data.month
    most_recent_year=most_recent_data.year
    most_recent_year_string=str(most_recent_year)
    year_string_today=most_recent_year_string
    if startyear<year:
        year_string=[str(i).zfill(2) for i in range(startyear, most_recent_year+1)]
    most_recent_data_string=most_recent_data.strftime('%Y-%m-%d')
    string_days=[str(i).zfill(2) for i in range(1, most_recent_data.day+1)]
    if most_recent_month==1:
        month_until_complete=12
        #year_string=str(year-1)
    else:
        month_until_complete=most_recent_month-1
    months=[str(i).zfill(2) for i in range(1, month_until_complete+1)]
    months_oldold=[str(i).zfill(2) for i in range(1, 12+1)]
    most_recent_month_string=str(most_recent_month).zfill(2)


    locations=[]
    for i in year_string:
        locations.append(wd+"era5_"+i+".nc")
    if most_recent_month==1:
        locations=locations[:-1]
        if path.exists(locations[-1])==True:
            os.remove(locations[-1]) # this is just to make sure we have the complete previous year stored
    locations.append(wd+"era5_newest_data.nc")
    
    
    for i in year_string:
            data_location=wd+"era5_"+i+".nc"
            newest_data_location=wd+"era5_newest_data.nc"
            if year_string_today==i:
                if most_recent_month>1:
                    c.retrieve(
                    'reanalysis-era5-single-levels',
                    {
                        'product_type': 'reanalysis',
                        'variable': variables,
                        'year': i,
                        'month': months,
                        'day': [
                            '01', '02', '03',
                            '04', '05', '06',
                            '07', '08', '09',
                            '10', '11', '12',
                            '13', '14', '15',
                            '16', '17', '18',
                            '19', '20', '21',
                            '22', '23', '24',
                            '25', '26', '27',
                            '28', '29', '30',
                            '31',
                        ],
                        'time': [
                            '00:00', '01:00', '02:00',
                            '03:00', '04:00', '05:00',
                            '06:00', '07:00', '08:00',
                            '09:00', '10:00', '11:00',
                            '12:00', '13:00', '14:00',
                            '15:00', '16:00', '17:00',
                            '18:00', '19:00', '20:00',
                            '21:00', '22:00', '23:00',
                        ],
                        'area': area,
                        'format': 'netcdf',
                    },
                    data_location)

                c.retrieve(
                'reanalysis-era5-single-levels',
                {
                    'product_type': 'reanalysis',
                    'variable': variables,
                    'year': str(most_recent_data.year),
                    'month': most_recent_month_string,
                    'day': string_days,
                    'time': [
                        '00:00', '01:00', '02:00',
                        '03:00', '04:00', '05:00',
                        '06:00', '07:00', '08:00',
                        '09:00', '10:00', '11:00',
                        '12:00', '13:00', '14:00',
                        '15:00', '16:00', '17:00',
                        '18:00', '19:00', '20:00',
                        '21:00', '22:00', '23:00',
                    ],
                    'area': area,
                    'format': 'netcdf',
                },
                newest_data_location)

            else:
                if path.exists(data_location)==False:
                    c.retrieve(
                    'reanalysis-era5-single-levels',
                    {
                        'product_type': 'reanalysis',
                        'variable': variables,
                        'year': i,
                        'month': months_oldold,
                        'day': [
                            '01', '02', '03',
                            '04', '05', '06',
                            '07', '08', '09',
                            '10', '11', '12',
                            '13', '14', '15',
                            '16', '17', '18',
                            '19', '20', '21',
                            '22', '23', '24',
                            '25', '26', '27',
                            '28', '29', '30',
                            '31',
                        ],
                        'time': [
                            '00:00', '01:00', '02:00',
                            '03:00', '04:00', '05:00',
                            '06:00', '07:00', '08:00',
                            '09:00', '10:00', '11:00',
                            '12:00', '13:00', '14:00',
                            '15:00', '16:00', '17:00',
                            '18:00', '19:00', '20:00',
                            '21:00', '22:00', '23:00',
                        ],
                        'area': area,
                        'format': 'netcdf',
                    },
                    data_location)
    merge_list=[] 
    
    locations=[]
    for file in rm_point(os.listdir(wd)):
        da=xr.open_dataset(file,engine="netcdf4")
        if len(da.dims)>3:
            #print(file)
            da=da.mean(("expver"))
            da.to_netcdf(file)
        locations.append(wd+file)
    #print(locations)
    era5=xr.open_mfdataset(locations,engine="netcdf4")
    if len(era5.dims)>3:
        era5=era5.mean(("expver"))
    return era5

In [ ]:
def plotting_annual_cycle(list_of_files,variables,endyear,plot_folder,color_newyear="red",region="CH",dataset="ERA5",ylabels=True,xlabels=False):
    copyright="@martin_wegmann"
    if (ylabels==True) and (xlabels==False):
    
       
        for variable in variables:
            fig, axs = plt.subplots(1, 1, figsize=(10, 5), dpi= 300)
            print("creating plot for "+dataset+" "+variable+" over "+region)
            for file in list_of_files:
                if len(file.time.dt.year.values)<365:
                    file[variable].plot(color=color_newyear,lw=2,label=str(endyear))
                else:
                    file[variable].plot(color="gray", alpha=0.4)
                units=file[variable].units
            title=dataset+" "+variable+" over "+region
            plt.title(title)
            plt.title(copyright,loc="right")
            plt.xlabel("")
            plt.ylabel("["+units+"]")
            plt.legend()
            #plt.xticks(rotation = 90)
            ax = plt.gca()
            ax.set_xticklabels(['01-01', '', '',"","","",'31-12'])
            ax.set_xticks([])

            save_name="annual_cycle_"+dataset+"_"+variable+"_"+region

            plt.savefig(plot_folder+save_name+"_nox.pdf")
            plt.savefig(plot_folder+save_name+"_nox.png")
            plt.show()
        
        
        
    if (ylabels==True) and (xlabels==True):    
        
        
        for variable in variables:
            fig, axs = plt.subplots(1, 1, figsize=(10, 5), dpi= 300)
            print("creating plot for "+dataset+" "+variable+" over "+region)
            for file in list_of_files:
                if len(file.time.dt.year.values)<365:
                    file[variable].plot(color=color_newyear,lw=2,label=str(endyear))
                else:
                    file[variable].plot(color="gray", alpha=0.4)
                units=file[variable].units
            title=dataset+" "+variable+" over "+region
            plt.title(title)
            plt.title(copyright,loc="right")
            plt.xlabel("")
            plt.ylabel("["+units+"]")
            plt.legend()
            #plt.xticks(rotation = 90)
            ax = plt.gca()
            ax.set_xticklabels(['01-01', '', '',"","","",'31-12'])
            

            save_name="annual_cycle_"+dataset+"_"+variable+"_"+region

            plt.savefig(plot_folder+save_name+".pdf")
            plt.savefig(plot_folder+save_name+".png")
            plt.show()
            
    if (ylabels==False) and (xlabels==False):    
        
        
        for variable in variables:
            fig, axs = plt.subplots(1, 1, figsize=(10, 5), dpi= 300)
            print("creating plot for "+dataset+" "+variable+" over "+region)
            for file in list_of_files:
                if len(file.time.dt.year.values)<365:
                    file[variable].plot(color=color_newyear,lw=2,label=str(endyear))
                else:
                    file[variable].plot(color="gray", alpha=0.4)
                units=file[variable].units
            title=dataset+" "+variable+" over "+region
            plt.title(title)
            plt.title(copyright,loc="right")
            plt.xlabel("")
            plt.ylabel("")
            plt.legend()
            #plt.xticks(rotation = 90)
            ax = plt.gca()
            ax.set_xticklabels(['01-01', '', '',"","","",'31-12'])
            ax.set_xticks([])
            ax.set_yticks([])

            save_name="annual_cycle_"+dataset+"_"+variable+"_"+region

            plt.savefig(plot_folder+save_name+"_noxy.pdf")
            plt.savefig(plot_folder+save_name+"_noxy.png")
            plt.show()
            
    if (ylabels==False) and (xlabels==True):    
        
        
        for variable in variables:
            fig, axs = plt.subplots(1, 1, figsize=(10, 5), dpi= 300)
            print("creating plot for "+dataset+" "+variable+" over "+region)
            for file in list_of_files:
                if len(file.time.dt.year.values)<365:
                    file[variable].plot(color=color_newyear,lw=2,label=str(endyear))
                else:
                    file[variable].plot(color="gray", alpha=0.4)
                units=file[variable].units
            title=dataset+" "+variable+" over "+region
            plt.title(title)
            plt.title(copyright,loc="right")
            plt.xlabel("")
            plt.ylabel("")
            plt.legend()
            #plt.xticks(rotation = 90)
            ax = plt.gca()
            ax.set_xticklabels(['01-01', '', '',"","","",'31-12'])
            #ax.set_xticks([])
            ax.set_yticks([])

            save_name="annual_cycle_"+dataset+"_"+variable+"_"+region

            plt.savefig(plot_folder+save_name+"_noy.pdf")
            plt.savefig(plot_folder+save_name+"_noy.png")
            plt.show()
    return 



In [ ]:
def plotting_annual_cumsum(list_of_files,variables,endyear,plot_folder,color_newyear="red",region="CH",dataset="ERA5",ylabels=True,xlabels=False):
    copyright="@martin_wegmann"
    if (ylabels==True) and (xlabels==False):
    
       
        for variable in variables:
            fig, axs = plt.subplots(1, 1, figsize=(10, 5), dpi= 300)
            print("creating plot for "+dataset+" "+variable+" over "+region)
            for file in list_of_files:
                if len(file.time.dt.year.values)<365:
                    file[variable].cumsum().plot(color=color_newyear,lw=2,label=str(endyear))
                else:
                    file[variable].cumsum().plot(color="gray", alpha=0.4)
                units=file[variable].units
            title=dataset+" "+variable+" over "+region
            plt.title(title)
            plt.title(copyright,loc="right")
            plt.xlabel("")
            plt.ylabel("["+units+"]")
            plt.legend()
            #plt.xticks(rotation = 90)
            ax = plt.gca()
            ax.set_xticklabels(['01-01', '', '',"","","",'31-12'])
            ax.set_xticks([])

            save_name="annual_cumsum_"+dataset+"_"+variable+"_"+region

            plt.savefig(plot_folder+save_name+"_nox.pdf")
            plt.savefig(plot_folder+save_name+"_nox.png")
            plt.show()
        
        
        
    if (ylabels==True) and (xlabels==True):    
        
        
        for variable in variables:
            fig, axs = plt.subplots(1, 1, figsize=(10, 5), dpi= 300)
            print("creating plot for "+dataset+" "+variable+" over "+region)
            for file in list_of_files:
                if len(file.time.dt.year.values)<365:
                    file[variable].cumsum().plot(color=color_newyear,lw=2,label=str(endyear))
                else:
                    file[variable].cumsum().plot(color="gray", alpha=0.4)
                units=file[variable].units
            title=dataset+" "+variable+" over "+region
            plt.title(title)
            plt.title(copyright,loc="right")
            plt.xlabel("")
            plt.ylabel("["+units+"]")
            plt.legend()
            #plt.xticks(rotation = 90)
            ax = plt.gca()
            ax.set_xticklabels(['01-01', '', '',"","","",'31-12'])
            

            save_name="annual_cumsum_"+dataset+"_"+variable+"_"+region

            plt.savefig(plot_folder+save_name+".pdf")
            plt.savefig(plot_folder+save_name+".png")
            plt.show()
            
    if (ylabels==False) and (xlabels==False):    
        
        
        for variable in variables:
            fig, axs = plt.subplots(1, 1, figsize=(10, 5), dpi= 300)
            print("creating plot for "+dataset+" "+variable+" over "+region)
            for file in list_of_files:
                if len(file.time.dt.year.values)<365:
                    file[variable].cumsum().plot(color=color_newyear,lw=2,label=str(endyear))
                else:
                    file[variable].cumsum().plot(color="gray", alpha=0.4)
                units=file[variable].units
            title=dataset+" "+variable+" over "+region
            plt.title(title)
            plt.title(copyright,loc="right")
            plt.xlabel("")
            plt.ylabel("")
            plt.legend()
            #plt.xticks(rotation = 90)
            ax = plt.gca()
            ax.set_xticklabels(['01-01', '', '',"","","",'31-12'])
            ax.set_xticks([])
            ax.set_yticks([])

            save_name="annual_cumsum_"+dataset+"_"+variable+"_"+region

            plt.savefig(plot_folder+save_name+"_noxy.pdf")
            plt.savefig(plot_folder+save_name+"_noxy.png")
            plt.show()
            
    if (ylabels==False) and (xlabels==True):    
        
        
        for variable in variables:
            fig, axs = plt.subplots(1, 1, figsize=(10, 5), dpi= 300)
            print("creating plot for "+dataset+" "+variable+" over "+region)
            for file in list_of_files:
                if len(file.time.dt.year.values)<365:
                    file[variable].cumsum().plot(color=color_newyear,lw=2,label=str(endyear))
                else:
                    file[variable].cumsum().plot(color="gray", alpha=0.4)
                units=file[variable].units
            title=dataset+" "+variable+" over "+region
            plt.title(title)
            plt.title(copyright,loc="right")
            plt.xlabel("")
            plt.ylabel("")
            plt.legend()
            #plt.xticks(rotation = 90)
            ax = plt.gca()
            ax.set_xticklabels(['01-01', '', '',"","","",'31-12'])
            #ax.set_xticks([])
            ax.set_yticks([])

            save_name="annual_cumsum_"+dataset+"_"+variable+"_"+region

            plt.savefig(plot_folder+save_name+"_noy.pdf")
            plt.savefig(plot_folder+save_name+"_noy.png")
            plt.show()
    return 




### define setup

In [ ]:
plt.style.use('dark_background')

In [ ]:
startyear=1951
today= datetime.today()
endyear=today.year

### define regional mask

In [ ]:
CH_index = regionmask.defined_regions.natural_earth_v5_0_0.countries_110.map_keys("Switzerland")



# download era5 data

In [ ]:
data_location="/Volumes/SPARK/era5/CH/"
plot_location="/Volumes/SPARK/era5/plots_CH/"
plot_folder=plot_location

In [ ]:
era_variables=[
            '2m_dewpoint_temperature', '2m_temperature','convective_available_potential_energy', 'maximum_2m_temperature_since_previous_post_processing', 'minimum_2m_temperature_since_previous_post_processing', 'snow_depth',
            'surface_solar_radiation_downwards', 'total_cloud_cover', 'total_precipitation', 'zero_degree_level','snowfall', 'snowmelt', 'vertical_integral_of_divergence_of_moisture_flux']

In [ ]:
era_area=[48, 5, 45,11,] # maxlat,minlon,minlat,maxlon

In [ ]:
era5_data=get_era5_hourly(startyear=startyear,area=era_area,variables=era_variables,wd=data_location)

### get a list of variables in the data we downloaded

In [ ]:

shortname_vars=list(era5_data.keys())
# rename coordiantes
era5_data=era5_data.rename({'longitude': 'lon','latitude': 'lat'})

### spatial overview of data

In [ ]:

era5_data.isel(time=0).d2m.plot(vmax=275,vmin=250)

### defining the masking

In [ ]:

mask = regionmask.defined_regions.natural_earth_v5_0_0.countries_110.mask(era5_data)
mask_CH = mask == CH_index
era5_data_CH = era5_data.where(mask_CH,drop=True)



### spatial overview of the masked data

In [ ]:

era5_data_CH.isel(time=0).d2m.plot(vmax=275,vmin=250)

### spatial weighting

In [ ]:

weights = np.cos(np.deg2rad(era5_data_CH.lat))
ts_era5_data_CH_weighted = era5_data_CH.weighted(weights).mean(dim=("lat", "lon"))




### resampling in time

In [ ]:

ts_era5_data_CH_weighted_daily=ts_era5_data_CH_weighted.resample(time="1D").mean()
ts_era5_data_CH_weighted_daily=nofeb(ts_era5_data_CH_weighted_daily)

years=np.unique(ts_era5_data_CH_weighted_daily.time.dt.year.values)

### create the same time axis for all years

In [ ]:
input_file=ts_era5_data_CH_weighted_daily

In [ ]:
list_of_files=[]


for year in years:
    print(str(year))
    if year==years[-1]:
        data=input_file.sel(time=slice(str(year)+"-01-01", input_file.time[-1]))
        data["time"] = ("time", input_file.sel(time=slice(str(startyear)+"-01-01", str(startyear)+"-12-31")).time.values[0:len(data.time.values)])
    else:
        data=input_file.sel(time=slice(str(year)+"-01-01", str(year)+"-12-31"))
        data["time"] = ("time", input_file.sel(time=slice(str(startyear)+"-01-01", str(startyear)+"-12-31")).time.values)
    list_of_files.append(data)
    

# plotting

In [ ]:
plotting_annual_cycle(list_of_files=list_of_files,variables=shortname_vars,endyear=endyear,plot_folder=plot_folder,color_newyear="red",region="CH",dataset="ERA5",ylabels=False,xlabels=False)

In [ ]:
plotting_annual_cumsum(list_of_files=list_of_files,variables=shortname_vars,endyear=endyear,plot_folder=plot_folder,color_newyear="red",region="CH",dataset="ERA5",ylabels=False,xlabels=False)